In [1]:
import socket
import threading

#dictionary to store authenticated users with IP Addresses
user_info ={}

#example
credentials = {
    "user1": "password1", 
    "user2": "password2",
    "admin": "admin125"
}

#encryption and decryption
def encrypt_message(message, key=3):
    encrypted = ''.join(chr((ord(char) - 32 - key) % 95+32) if char.isprintable() else char for char in message)
    return encrypted

def decrypt_message(message, keys=3):
    decrypted = ''.join(chr((ord (char) - 32 - key)% 95 + 32) if char.isprintable() else char for char in message)
    return decrypted

#handle single client
def handle_client(server_socket):
    while True:
        try:
            data, client_address = server_socket.recvfrom(2048)
            message = data.decode()
            
            #check if client has used authentication
            if client_address not in user_info:
                username, password = message.split(":")
                if username in credentials and credentials[username] == password:
                    #add client user dictionary
                    user_info[client_address] = {"username": username}
                    response = "AUTH_SUCCESS"
                    print(f"{username} Anthentication Successfully.")
                else:
                    response = "AUTH_UNSUCCESSFUL"
                    print(f"Authentication is unsuccessful for {client_address}.")
                server_socket.sendto(response.encode(), client_address)
                continue
            
            #unauthenticated, broadcast encrypted message
            username = user_info[client_address]["username"]
            decrypt_message = decrypt_message(message)
            print(f"{username} ({client_address}): {decrypt_message}")
            
            #re-encrypt
            encrypted_message = encrypt_message(f"{username}: {decrypted_message}")
            for client in user_info.keys ():
                if client != client_address:
                    server_socket.sendto(encrypted_message.encode(), client)
        except Exception as e:
            print(f"client error handling {client_address}: {e}")
            break
        
    #setup udp server
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server_socket.bind(('localhost', 65433))
    print("Server is working..")
    
    #start clinet handler
    thread = threading.thread(target=handle_client, args=(server_socket,))
    thread.daemon = True
    thread.start()
    
    print("Press ctrl+c to end the server")
    try:
        thread.join()
    except KeyboardInterrupt:
        print("\nServer shutting down...")
    finally:


     server_socket.close()
            
            
            